In [21]:
from collections import defaultdict
import json, math, glob
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.express as px
from tqdm import tqdm
import math
import os

In [22]:
records = []
for fname in glob.glob(f"data/*.jsonl"):
    with open(fname, 'rt') as f:
        records.extend([json.loads(l) for l in f.readlines()])

eval_results = pd.DataFrame(records)
display(eval_results.describe())



print(set(eval_results['benchmark_id']))

benchmark_id = 'mbpp+'
result = eval_results[eval_results['benchmark_id'] == benchmark_id]
display(result)

# figs = basic_analysis(result, benchmark_id)


,pass1
count,73280.000000
mean,0.470794
std,0.474130
min,0.000000
25%,0.000000
50%,0.300000
75%,1.000000
max,1.000000


{'CRUXEval-output', 'mbpp+', 'CRUXEval-input', 'humaneval+'}


,benchmark_id,model,example_id,pass1,hyperparams,groundtruth,raw_genereation,sanitized_generation,fail_base,fail_plus,base_fail_tests,plus_fail_tests
60428,mbpp+,CohereForAI--c4ai-command-r-plus,Mbpp/2,1.0,temp0.0,"\ndef similar_elements(test_tup1, test_tup2):\...","def similar_elements(list1: list, list2: list)...","def similar_elements(list1: list, list2: list)...",False,False,[],[]
60429,mbpp+,CohereForAI--c4ai-command-r-plus,Mbpp/3,1.0,temp0.0,\nimport math\ndef is_not_prime(n):\n if n ...,"def is_not_prime(n: int) -> bool:\n """"""Chec...","def is_not_prime(n: int) -> bool:\n """"""Chec...",False,False,[],[]
60430,mbpp+,CohereForAI--c4ai-command-r-plus,Mbpp/4,1.0,temp0.0,\nimport heapq as hq\ndef heap_queue_largest(n...,import heapq\n\ndef heap_queue_largest(numbers...,import heapq\ndef heap_queue_largest(numbers: ...,False,False,[],[]
60431,mbpp+,CohereForAI--c4ai-command-r-plus,Mbpp/6,0.0,temp0.0,\ndef is_Power_Of_Two(x: int): \n return x ...,"def differ_At_One_Bit_Pos(n, m):\n return (...","def differ_At_One_Bit_Pos(n, m):\n return (...",True,True,[],"[[-1, 0]]"
60432,mbpp+,CohereForAI--c4ai-command-r-plus,Mbpp/7,0.0,temp0.0,\nimport re\ndef find_char_long(text):\n retu...,def find_char_long(s):\n return [word for w...,def find_char_long(s):\n return [word for w...,False,True,[],[[This is a very long string with no words tha...
...,...,...,...,...,...,...,...,...,...,...,...,...
73275,mbpp+,xwincoder-34b,Mbpp/805,0.0,temp0.0,"\ndef max_sum_list(lists):\n return max(lists,...",def max_sum_list(list_of_lists):\n max_sum ...,def max_sum_list(list_of_lists):\n max_sum ...,False,True,[],"[[[[-10, -20, -30], [-40, -50, -60], [-70, -80..."
73276,mbpp+,xwincoder-34b,Mbpp/806,0.0,temp0.0,\ndef max_run_uppercase(test_str):\n cnt = 0\...,def max_run_uppercase(s):\n max_run = 0\n ...,def max_run_uppercase(s):\n max_run = 0\n ...,False,True,[],[[Aaa]]
73277,mbpp+,xwincoder-34b,Mbpp/807,1.0,temp0.0,\ndef first_odd(nums):\n first_odd = next((el...,def first_odd(numbers):\n for number in num...,def first_odd(numbers):\n for number in num...,False,False,[],[]
73278,mbpp+,xwincoder-34b,Mbpp/808,1.0,temp0.0,"\ndef check_K(test_tup, K):\n return K in tes...","def check_K(tup, k):\n return k in tup\n\n#...","def check_K(tup, k):\n return k in tup",False,False,[],[]


In [23]:
def pass1_to_battle(result: pd.DataFrame):
    pa = pd.merge(result, result, on=['example_id'], suffixes=["_a", "_b"], how='outer')

    awins = (pa['pass1_a'] > 0) & (pa['pass1_b'] == 0)
    bwins = (pa['pass1_a'] == 0) & (pa['pass1_b'] > 0)
    ties_neither = (pa['pass1_a'] == 0) & (pa['pass1_b'] == 0)
    ties_both = (pa['pass1_a'] > 0) & (pa['pass1_b'] > 0)
    # pa[['winner']][awins] = 'model_a' 
    pa['winner'] = 'a'
    pa.loc[awins, 'winner'] = 'model_a'
    pa.loc[bwins, 'winner'] = 'model_b'
    pa.loc[ties_neither, 'winner'] = 'neither'
    pa.loc[ties_both, 'winner'] = 'both'
    return pa 

battles = pass1_to_battle(result)
accs = result.groupby('model').agg('mean').reset_index().sort_values(by='pass1', ascending=False)
display(accs)

display(battles.describe())

/tmp/ipykernel_2154564/3357203197.py:17: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,model,pass1
17,gpt-4-1106-preview,0.732804
18,meta-llama-3-70b-instruct,0.690476
26,opencodeinterpreter-ds-33b,0.685185
32,white-rabbit-neo-33b-v1,0.669312
27,opencodeinterpreter-ds-6.7b,0.664021
12,deepseek-coder-6.7b-instruct,0.656085
3,bigcode--starcoder2-15b-instruct-v0.1,0.650794
33,xwincoder-34b,0.648148
1,HuggingFaceH4--starchat2-15b-v0.1,0.645503
22,mixtral-8x22b-instruct-v0.1,0.642857


,pass1_a,pass1_b
count,436968.000000,436968.000000
mean,0.541394,0.541394
std,0.498284,0.498284
min,0.000000,0.000000
25%,0.000000,0.000000
50%,1.000000,1.000000
75%,1.000000,1.000000
max,1.000000,1.000000


In [24]:
def visualize_battle_count(battles, title, show_num_models=30):
    ptbl = pd.pivot_table(battles, index="model_a", columns="model_b", aggfunc="size",
                          fill_value=0)
    battle_counts = ptbl
    ordering = battle_counts.sum().sort_values(ascending=True).index
    ordering = ordering[:show_num_models]
    fig = px.imshow(battle_counts.loc[ordering, ordering],
                    title=title, text_auto=True)
    fig.update_layout(xaxis_title="Model B",
                      yaxis_title="Model A",
                      xaxis_side="top", height=800, width=800,
                      title_y=0.07, title_x=0.5,
                      font=dict(size=10))
    fig.update_traces(hovertemplate=
                      "Model A: %{y}<br>Model B: %{x}<br>Count: %{z}<extra></extra>")
    return fig

battles_no_ties = battles[battles["winner"].str.contains("model_")]
fig = visualize_battle_count(battles_no_ties, "No ties")
fig

In [50]:

def compute_pairwise_win_fraction(battles, max_num_models=30):

    num_battles_ptbl = pd.pivot_table(battles,
        index="model_a", columns="model_b", aggfunc="size", fill_value=0)
    
    a_win = pd.pivot_table(
        battles[battles['winner'] == "model_a"],
        index="model_a", columns="model_b", aggfunc="size", fill_value=0) / num_battles_ptbl  
    
    b_win = pd.pivot_table(
        battles[battles['winner'] == "model_b"],
        index="model_a", columns="model_b", aggfunc="size", fill_value=0) / num_battles_ptbl

    neither = pd.pivot_table(
        battles[battles['winner'] == "neither"],
        index="model_a", columns="model_b", aggfunc="size", fill_value=0) / num_battles_ptbl
    
    both = pd.pivot_table(
        battles[battles['winner'] == "both"],
        index="model_a", columns="model_b", aggfunc="size", fill_value=0) / num_battles_ptbl 

    
    # 
    prop_wins = a_win.mean(axis=1).sort_values(ascending=False)
    prop_wins = prop_wins[:max_num_models]
    sort_keys = list(prop_wins.keys())
    
    return tuple(x.loc[sort_keys, sort_keys] for x in [a_win, b_win, neither, both])

def compute_pvalues(battles, max_num_models=100):
    # Times each model wins as Model A
    a_win_ptbl = pd.pivot_table(
        battles[battles['winner'] == "model_a"],
        index="model_a", columns="model_b", aggfunc="size", fill_value=0)

    # Table counting number of A-B pairs
    num_battles_ptbl = pd.pivot_table(battles,
        index="model_a", columns="model_b", aggfunc="size", fill_value=0)

    # Computing the proportion of wins for each model as A and as B
    # against all other models
    row_beats_col_freq = (
        (a_win_ptbl) /
        (num_battles_ptbl)
    )
    # display(mcnemar)
    # Arrange ordering according to proprition of wins
    prop_wins = row_beats_col_freq.mean(axis=1).sort_values(ascending=False)
    prop_wins = prop_wins[:max_num_models]
    model_names = list(prop_wins.keys())
    row_beats_col = row_beats_col_freq.loc[model_names, model_names]

    wins = a_win_ptbl.loc[model_names, model_names]
    diffs = (wins - wins.T)
    sums = (wins + wins.T)
    suf_stats = (pd.concat([wins, wins - wins.T, wins + wins.T])
    .stack(dropna=False)
    .groupby(level=[0,1])
    .apply(tuple)
    .unstack()
    ).loc[model_names, model_names]
    chi2 = suf_stats.applymap(lambda x: 1 if x[2] == 0 else 1 - stats.chi2.cdf( (np.abs(x[1]) - 1)**2 / (x[2]), 1))
    binom = suf_stats.applymap(lambda x: stats.binomtest(x[0], x[2], p=0.5).pvalue if x[2] > 0 else 1)
    return row_beats_col, binom, diffs, sums, chi2 

def visualize_pairwise_win_fraction(battles, title, max_num_models=30):
    a_win, b_win, neither, both = compute_pairwise_win_fraction(battles, max_num_models)

    fig = px.imshow(a_win, color_continuous_scale='RdBu',
                    text_auto=".2f", title=title)
    fig.update_layout(xaxis_title=" Model B: Loser",
                  yaxis_title="Model A: Winner",
                  xaxis_side="top", height=900, width=900,
                  title_y=0.07, title_x=0.5)

    sort_keys = a_win.keys()  
    extra_info = (pd.concat([a_win, b_win, neither, both])
    .stack(dropna=False)
    .groupby(level=[0,1])
    .apply(tuple).apply(lambda t: tuple(s if not math.isnan(s) else 'nan' for s in t))
    .unstack()
    ).loc[sort_keys, sort_keys]
    fig.update_traces(customdata=extra_info, hovertemplate="<br>".join(
        [
            "A Wins: %{customdata[0]:.3f}",
            "B Wins: %{customdata[1]:.3f}",
            "Neither: %{customdata[2]:.3f}",
            "Both: %{customdata[3]:.3f}",
        ])
    )

    return fig

def visualize_pvalues(battles, title, max_num_models=30):
    row_beats_col, pvalue, diffs, sums, chi2 = compute_pvalues(battles, max_num_models)
    fig = px.imshow(pvalue,
                    text_auto=".2f", title=title)
    fig.update_layout(xaxis_title=" Model B",
                  yaxis_title="Model A",
                  xaxis_side="top", height=900, width=900,
                  title_y=0.07, title_x=0.5)

    sort_keys = row_beats_col.keys() 
    extra_info = (pd.concat([row_beats_col, pvalue, diffs, sums, chi2])
        .stack(dropna=False)
        .groupby(level=[0,1])
        .apply(tuple).apply(lambda t: tuple(s if not math.isnan(s) else 'nan' for s in t))
        .unstack()
    ).loc[sort_keys, sort_keys]
    
    # display(row_beats_col)
    # display(extra_info)
    fig.update_traces(customdata=extra_info, hovertemplate=
        "<br>".join([
        "Model A: %{y}",
        "Model B: %{x}", 
        "binom p-value: %{customdata[1]:.4f}",
        "chi2 p-value: %{customdata[4]:.4f}",
        "A_win frac.: %{customdata[0]:.2f}",
        "A_win - B_win: %{customdata[2]}",
        "A_win + B_win: %{customdata[3]}",
        ])  + '<extra></extra>')

    return fig


fig_pvalues = visualize_pvalues(battles_no_ties, f'p-values {benchmark_id}', max_num_models=60)
fig_pairwin = visualize_pairwise_win_fraction(battles, 'win_rates', max_num_models=60)
display(fig_pairwin)
display(fig_pvalues)
template_path=r"report_template.html"
output_path = rf"reports/{benchmark_id}.html"

sections = {
    "pairwise wins": fig_pairwin.to_html(full_html=False),
    "p-values": fig_pvalues.to_html(full_html=False),
}
#consider also defining the include_plotlyjs parameter to point to an external Plotly.js as described above


In [51]:
row_beats_col, pvalue, diffs, sums, chi2 = compute_pvalues(battles, 100)
df = pvalue.reset_index()
df_pval = df.melt(id_vars='model_a', value_vars=list(df.columns[1:]), var_name='model_b', value_name='p_value')
df = diffs.reset_index()
df_diffs = df.melt(id_vars='model_a', value_vars=list(df.columns[1:]), var_name='model_b', value_name='diff')
benchmark_size = len(set(result['example_id']))
df_diffs['delta(A, B)'] = df_diffs['diff'].abs() / benchmark_size
df_diffs['models'] = df_diffs['model_a'] + ' vs. ' + df_diffs['model_b']

df = df_pval.merge(df_diffs, on=['model_a', 'model_b'])
display(df)
sections['fig_deltas'] = px.scatter(df, x='delta(A, B)', y='p_value', hover_data='models').to_html(full_html=False)


,model_a,model_b,p_value,diff,"delta(A, B)",models
0,gpt-4-1106-preview,gpt-4-1106-preview,1.000000,0,0.000000,gpt-4-1106-preview vs. gpt-4-1106-preview
1,opencodeinterpreter-ds-33b,gpt-4-1106-preview,0.047393,-18,0.047619,opencodeinterpreter-ds-33b vs. gpt-4-1106-preview
2,meta-llama-3-70b-instruct,gpt-4-1106-preview,0.055897,-16,0.042328,meta-llama-3-70b-instruct vs. gpt-4-1106-preview
3,white-rabbit-neo-33b-v1,gpt-4-1106-preview,0.007905,-24,0.063492,white-rabbit-neo-33b-v1 vs. gpt-4-1106-preview
4,bigcode--starcoder2-15b-instruct-v0.1,gpt-4-1106-preview,0.000639,-31,0.082011,bigcode--starcoder2-15b-instruct-v0.1 vs. gpt-...
...,...,...,...,...,...,...
1151,mistral-7b,gemma-2b,0.000535,30,0.079365,mistral-7b vs. gemma-2b
1152,codet5p-2b,gemma-2b,0.105342,15,0.039683,codet5p-2b vs. gemma-2b
1153,gemma-7b-it,gemma-2b,0.699175,-5,0.013228,gemma-7b-it vs. gemma-2b
1154,codegen-2b,gemma-2b,0.510410,7,0.018519,codegen-2b vs. gemma-2b


In [63]:
from jinja2 import Template
with open(output_path, "w", encoding="utf-8") as output_file:
    with open(template_path) as template_file:
        j2_template = Template(template_file.read())
        output_file.write(j2_template.render({'benchmark_id': benchmark_id, 'sections': sections}))

In [70]:
def compute_mle_elo(
    df, SCALE=400, BASE=10, INIT_RATING=1000
):
    from sklearn.linear_model import LogisticRegression
    ptbl_a_win = pd.pivot_table(
        df[df["winner"] == "model_a"],
        index="model_a",
        columns="model_b",
        aggfunc="size",
        fill_value=0,
    )
    # if no tie, create a zero matrix
    if sum(df["winner"].isin(["both", "both"])) == 0:
        ptbl_tie = pd.DataFrame(0, index=ptbl_a_win.index, columns=ptbl_a_win.columns)
    else:
        ptbl_tie = pd.pivot_table(
            df[df["winner"].isin(["both", "neither"])],
            index="model_a",
            columns="model_b",
            aggfunc="size",
            fill_value=0,
        )
        ptbl_tie = ptbl_tie + ptbl_tie.T

    ptbl_b_win = pd.pivot_table(
        df[df["winner"] == "model_b"],
        index="model_a",
        columns="model_b",
        aggfunc="size",
        fill_value=0,
    )
    ptbl_win = ptbl_a_win * 2 + ptbl_b_win.T * 2 + ptbl_tie
    # display(ptbl_win)
    # display(ptbl_tie)

    models = pd.Series(np.arange(len(ptbl_win.index)), index=ptbl_win.index)

    p = len(models)
    X = np.zeros([p * (p - 1) * 2, p])
    Y = np.zeros(p * (p - 1) * 2)

    cur_row = 0
    sample_weights = []
    for m_a in ptbl_win.index:
        for m_b in ptbl_win.columns:
            if m_a == m_b:
                continue
            # if nan skip
            if math.isnan(ptbl_win.loc[m_a, m_b]) or math.isnan(ptbl_win.loc[m_b, m_a]):
                continue
            X[cur_row, models[m_a]] = +math.log(BASE)
            X[cur_row, models[m_b]] = -math.log(BASE)
            Y[cur_row] = 1.0
            sample_weights.append(ptbl_win.loc[m_a, m_b])

            X[cur_row + 1, models[m_a]] = math.log(BASE)
            X[cur_row + 1, models[m_b]] = -math.log(BASE)
            Y[cur_row + 1] = 0.0
            sample_weights.append(ptbl_win.loc[m_b, m_a])
            cur_row += 2
    X = X[:cur_row]
    Y = Y[:cur_row]
    # fig = px.imshow(X)
    # display(fig)
    # print(Y, Y.shape)

    lr = LogisticRegression(fit_intercept=False, penalty=None, tol=1e-6)
    lr.fit(X, Y, sample_weight=sample_weights)
    elo_scores = SCALE * lr.coef_[0] + INIT_RATING
    if "gpt-3.5-turbo-0613" in models.index:
        elo_scores += 1000 - elo_scores[models["gpt-3.5-turbo-0613"]]
    return pd.Series(elo_scores, index=models.index).sort_values(ascending=False)

model_elos = compute_mle_elo(battles_no_ties)

In [72]:
pair_stats = compute_pairwise_win_fraction(battles)
a_win = pair_stats[0]
win_rates = a_win.mean(axis=1).sort_values(ascending=False)
win_elo = win_rates.to_frame(name='win_rate').join(model_elos.to_frame(name='elo')).reset_index()
all_stats = win_elo.merge(accs, left_on='model_a', right_on='model')[['model', 'pass1', 'win_rate', 'elo']]
sections['Ratings table'] = all_stats.to_html(float_format='%10.3f')
px.scatter(all_stats, x='elo', y='win_rate')

In [ ]:
import plotly.graph_objects as go

records = []
len_data = len(set(result['example_id']))
for i in np.arange(len_data):
    current_id = f'Mbpp/{i}'
    example_data = result[result['example_id'] == current_id][['model', 'pass1']]
    fit_data = example_data.merge(model_elos.to_frame(name='elo'), left_on = 'model', right_on = 'model_a')
    # fit_data['result'] = fit_data['result']
    from sklearn.linear_model import LogisticRegression
    lr = LogisticRegression()
    fit_data['correct'] = np.where(fit_data['pass1'] > 0, 1, 0)
    if all(fit_data['correct'].to_numpy() == 0) or all(fit_data['correct'].to_numpy() == 1):
        mean = 1 if all(fit_data['correct'].to_numpy() == 1) else 0
        records.append({
            'sample_id': f'sample_{i}_input',
            'elo': -3500 if mean > 0 else 3500,
            'elo_var': 0,
            'score': 1,
            'acc': mean,
        })
        continue
    else:
        Xd = fit_data['elo'].to_numpy().reshape(-1, 1)
        yd = fit_data['correct'].to_numpy() 
        lrm = lr.fit(fit_data['elo'].to_numpy().reshape(-1, 1), fit_data['correct'].to_numpy())
        X = list(range(500, 1500, 10))
        y = [lrm.predict_proba([[x]])[0, 1] for x in X]
        elo = -lrm.intercept_[0] / lrm.coef_[0][0]
        elo_var = 1/lrm.coef_[0][0]
        score = lrm.score(Xd, yd)
        records.append({
            'sample_id': f'sample_{i}_input',
            'elo': elo,
            'elo_var': elo_var,
            'score': score,
            'acc': fit_data['correct'].to_numpy().mean()
        })
    if 5 < elo_var < 50:
        print(elo, elo_var)
        print(fit_data)
        fig1 = px.scatter(fit_data, x='elo', y='pass1', hover_name='model', title=current_id)
        fig1.update_traces(textposition='top center')
        fig2 = px.line(x=X, y=y)
        fig3 = go.Figure(data=fig1.data + fig2.data, layout=fig1.layout)
        display(fig3)
    # display(fig3)


In [ ]:
import plotly.express as px
df = px.data.tips()

df = pd.DataFrame.from_records(records)
display(df)
fig = px.histogram(df, x='acc', marginal="rug")
# fig.update_traces(xbins=dict( # bins used for histogram
#         start=0.0,
#         end=1.1,
#         size=0.1,
#     ))
print(df[df['acc'] < 0.2].describe())
# print(18+8+8+12+17+10+17+17+14+20+23)
fig

In [ ]:
import sys
sys.path.append('/private/home/sida/git/CodeGen/scripts/sida/arena/data/evalplus')


from evalplus.data import get_mbpp_plus, write_jsonl

samples = [
    dict(task_id=task_id, problem=problem['prompt'])
    for task_id, problem in get_mbpp_plus().items()
]
print(samples[77])
df = pd.DataFrame(samples)
# weird ones 77, 233
print(df[df['task_id'] == 'Mbpp/233']['problem'].iloc[0])